## Imports

In [1]:
import numpy as np
import pandas as pd
import sbibm
import torch
import altair_saver
import tensorflow_probability as tfp

from sbibm.metrics import c2st
from sbibm.visualisation import fig_posterior
from sbibm.metrics import c2st

## Helper function

In [2]:
def thin(X, length_out):
    assert length_out < X.shape[0]
    n = X.shape[0]
    keep = np.round(np.linspace(1, n, num = length_out)) - 1
    keep = keep.astype(int)
    return X[keep, :]

## Posterior plots

In [54]:
task_names = ["gaussian_linear", "gaussian_linear_uniform", "gaussian_mixture", "bernoulli_glm"]
algorithm_names = ["rula", "bsl"]

plot_n = 1000
for task_name in task_names:
    for algorithm_name in algorithm_names:
        posterior_samples = np.genfromtxt(f"./samples/{task_name}_{algorithm_name}.txt")
        posterior_samples = thin(posterior_samples, plot_n)
        posterior_samples = torch.tensor(posterior_samples)        
        
        n_params = posterior_samples.shape[1] 
        fig_size = 400-35*n_params
        scatter_size = 3.5+0.05*n_params

        fig = fig_posterior(
            task_name=task_name,
            observation=1,
            samples_tensor = posterior_samples,
            num_samples = plot_n,
            config = "streamlit",
            height = fig_size,
            width = fig_size,
            scatter_size = scatter_size,
            samples_name = algorithm_name
        )
        altair_saver.save(fig, f"./plots/{task_name}_{algorithm_name}.html")

    

## Performance metrics
We will compute the classification accuracy twice. Once on all the samples (3000), then once on a thinned sample of 300 points. The size of the reference is matched to the size of the data to make sure the neural network did not have issues with the biased class sizes.

In [55]:
metrics = {
    "task": [],
    "algorithm":  [],
    "min_ess": [],
    "mean_ess": [],
    "max_ess": [],
    "c2st_all": [],
    "c2st_thinned": []
}

for task_name in task_names:
    for algoritm_name in algorithm_names:
        print(f"{task_name}: {algoritm_name}")
        posterior_samples = torch.tensor(np.genfromtxt(f"./samples/{task_name}_{algoritm_name}.txt"))
        task = sbibm.get_task(task_name)
        reference_samples = task.get_reference_posterior_samples(num_observation=1)
        
        ess = tfp.mcmc.effective_sample_size(posterior_samples)

        X = posterior_samples
        Y = reference_samples[1:X.shape[0], :]
        c2st_all = c2st(X, Y)[0].item()
        
        X = thin(posterior_samples, 300)
        Y = reference_samples[1:X.shape[0], :]
        c2st_thinned = c2st(X, Y)[0].item()      
        
        metrics["task"].append(task_name)
        metrics["algorithm"].append(algoritm_name)
        metrics["min_ess"].append(np.min(ess))
        metrics["mean_ess"].append(np.mean(ess))
        metrics["max_ess"].append(np.max(ess))
        metrics["c2st_all"].append(c2st_all)
        metrics["c2st_thinned"].append(c2st_thinned)


gaussian_linear: rula
gaussian_linear: bsl
gaussian_linear_uniform: rula
gaussian_linear_uniform: bsl
gaussian_mixture: rula
gaussian_mixture: bsl
bernoulli_glm: rula
bernoulli_glm: bsl


In [56]:
df = pd.DataFrame(metrics)
df

,task,algorithm,min_ess,mean_ess,max_ess,c2st_all,c2st_thinned
0,gaussian_linear,rula,85.013037,118.945659,153.504177,0.724957,0.552591
1,gaussian_linear,bsl,35.533856,64.527633,125.810365,0.940156,0.661078
2,gaussian_linear_uniform,rula,60.307508,155.880855,296.382084,0.777798,0.577703
3,gaussian_linear_uniform,bsl,18.259304,32.361557,49.390491,0.942655,0.789622
4,gaussian_mixture,rula,222.964385,224.768997,226.573609,0.770626,0.774566
5,gaussian_mixture,bsl,120.105172,152.755511,185.405850,0.797132,0.757941
6,bernoulli_glm,rula,167.468600,261.934692,409.087939,0.690614,0.671176
7,bernoulli_glm,bsl,27.037251,61.608195,154.650830,0.928154,0.701317


### Combine this with other results

In [57]:
rula_df = pd.read_csv("results/rula.csv")
rula_df["algorithm"] = "rula"
rula_df["acceptance_rate"] = 1.

bsl_df = pd.read_csv("results/bsl.csv")
bsl_df["algorithm"] = "bsl"

In [58]:
df = df.merge(rula_df.append(bsl_df), on = ["task", "algorithm"])
df

,task,algorithm,min_ess,mean_ess,max_ess,c2st_all,c2st_thinned,run_time,acceptance_rate
0,gaussian_linear,rula,85.013037,118.945659,153.504177,0.724957,0.552591,1648.154514,1.000000
1,gaussian_linear,bsl,35.533856,64.527633,125.810365,0.940156,0.661078,1049.238019,0.139333
2,gaussian_linear_uniform,rula,60.307508,155.880855,296.382084,0.777798,0.577703,1580.158199,1.000000
3,gaussian_linear_uniform,bsl,18.259304,32.361557,49.390491,0.942655,0.789622,1040.553446,0.402333
4,gaussian_mixture,rula,222.964385,224.768997,226.573609,0.770626,0.774566,734.566935,1.000000
5,gaussian_mixture,bsl,120.105172,152.755511,185.405850,0.797132,0.757941,621.533106,0.516667
6,bernoulli_glm,rula,167.468600,261.934692,409.087939,0.690614,0.671176,1639.482386,1.000000
7,bernoulli_glm,bsl,27.037251,61.608195,154.650830,0.928154,0.701317,1095.160105,0.193000


In [59]:
df.to_csv("results/general_performance.csv")